Make Prediction through the trained model 

In [ ]:
#now importing library 

In [49]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sb 
from sklearn.preprocessing import OneHotEncoder , LabelEncoder
import pickle

In [50]:
# now import all pickle file which we need to to standardscaler and onehotencoder , labelEncoder
with open('label_encoder_gender.pkl', 'rb') as file:
    label_gender_encoder=pickle.load(file)

# now one_hot 
with open('one_hot_encoder_geo.pkl' ,'rb') as file:
    one_hot_encoder=pickle.load(file)
    
# now starndardization
with open('Standard_data.pkl' ,'rb') as file:
    standard_data= pickle.load(file)

In [51]:
# now load the model
from tensorflow.keras.models import load_model
model=load_model('ann_model.h5')

In [52]:
# Input data 
input_data=[{
    "CreditScore": 528,
    "Geography": "Germany",
    "Gender": "Male",
    "Age": 31,
    "Tenure": 6,
    "Balance": 0.00,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 0,
    "EstimatedSalary": 80181.12
  }]

In [53]:
# Since our model need only numerical value so, we need to first convert the input value into teh numerical form 
# first convert the gender to its numerical value
# since input data is a dictionary then first we have also convert it into dataframe
input_data_df=pd.DataFrame(input_data)
input_data_df.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,528,Germany,Male,31,6,0.0,2,1,0,80181.12


In [54]:
# now changes
input_data_df['Gender']=label_gender_encoder.transform([input_data_df['Gender']])

d:\ANN_Classification\first_ann_model\lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [55]:
input_data_df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,528,Germany,1,31,6,0.0,2,1,0,80181.12


In [56]:
# now change the Geography part to the numerical by one hot encoder
one_hot_encoded=one_hot_encoder.transform(input_data_df[['Geography']]).toarray()
geo_columns=one_hot_encoder.get_feature_names_out(['Geography'])

In [57]:
# now create a dataframe 
geo_dataframe=pd.DataFrame(one_hot_encoded ,columns=geo_columns)

In [58]:
geo_dataframe.head()

,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0


In [59]:
# now add this to the the main input dataframe
input_data_df_updated=pd.concat([input_data_df,geo_dataframe ],axis=1)

In [60]:
input_data_df_updated.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,528,Germany,1,31,6,0.0,2,1,0,80181.12,0.0,1.0,0.0


In [61]:
# remove teh georapy
input_data_df_updated=input_data_df_updated.drop(['Geography'] ,axis=1)

In [62]:
input_data_df_updated

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,528,1,31,6,0.0,2,1,0,80181.12,0.0,1.0,0.0


In [63]:
input_data_df_updated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        1 non-null      int64  
 1   Gender             1 non-null      int64  
 2   Age                1 non-null      int64  
 3   Tenure             1 non-null      int64  
 4   Balance            1 non-null      float64
 5   NumOfProducts      1 non-null      int64  
 6   HasCrCard          1 non-null      int64  
 7   IsActiveMember     1 non-null      int64  
 8   EstimatedSalary    1 non-null      float64
 9   Geography_France   1 non-null      float64
 10  Geography_Germany  1 non-null      float64
 11  Geography_Spain    1 non-null      float64
dtypes: float64(5), int64(7)
memory usage: 224.0 bytes


In [65]:
# now standardized the 
standardized_input=standard_data.transform(input_data_df_updated)

In [66]:
standardized_input

array([[-1.29028926,  0.91055421, -0.75193509,  0.35138484, -1.22205481,
         0.81966266,  0.64598061, -1.03016917, -0.35806731, -1.00171576,
         1.73734559, -0.57779016]])

In [67]:
# As we get the input in the standardized form then now we have to give to the model to predict

In [68]:
predicted_value=model.predict(standardized_input)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 399ms/step


In [69]:
print(predicted_value)

[[0.07053411]]


In [71]:
# the out put is in the probability form 
# let's this in the form of 

if predicted_value<0.5:
    print("Coustomer will not leave the Bank ")
else:
    print("Customer will leave the bank")

Coustomer will not leave the Bank 
